In [1]:
import numpy as np
import pandas as pd

from multipyles import multipyles, read_from_dft

pd.set_option('display.float_format', '{:.3f}'.format)

# Reads the occupation matrix from VASP

In [2]:
with open(f'tests/Cr2O3_benchmark_vasp/OUTCAR', 'r') as file:
    occupation_per_site = read_from_dft.read_densmat_from_vasp(file, select_atoms=(1, 2, 3, 4))

In [3]:
occupation_per_site.shape

(4, 5, 5, 2, 2)

The data has the shape n_atoms x n_orb x n_orb x n_spin x n_spin. Always: n_spin=2, n_orb=2l+1.

Printing the density matrix for the first Re atom and spin (up/up):

In [4]:
with np.printoptions(precision=3, suppress=True, floatmode='fixed'):
    print(occupation_per_site[0, :, :, 0, 0])

[[ 0.650+0.000j -0.058+0.002j -0.000+0.000j  0.304+0.000j  0.000-0.007j]
 [-0.058-0.002j  0.441+0.000j  0.000+0.000j -0.000-0.007j  0.304-0.000j]
 [ 0.000+0.000j  0.000+0.000j  0.864+0.000j  0.000+0.000j -0.000+0.000j]
 [ 0.304-0.000j  0.000+0.007j  0.000+0.000j  0.441+0.000j  0.058+0.002j]
 [ 0.000+0.007j  0.304+0.000j  0.000+0.000j  0.058-0.002j  0.650+0.000j]]


# Runs multipole code

In [5]:
results, l = multipyles.calculate(occupation_per_site, verbose=False)

Angular momentum of density matrix is l = 2
Transforming from cubic to spherical harmonics


In [6]:
results

,species,atom,nu,l1,l2,k,p,r,t,value
0,1,0,0,2,2,0,0,0,0,3.507+0.000j
1,1,1,0,2,2,0,0,0,0,3.507+0.000j
2,1,2,0,2,2,0,0,0,0,3.507+0.000j
3,1,3,0,2,2,0,0,0,0,3.507+0.000j
4,1,0,1,2,2,0,0,0,0,0.000+0.000j
...,...,...,...,...,...,...,...,...,...,...
795,1,3,1,2,2,4,1,5,1,0.001-0.001j
796,1,3,1,2,2,4,1,5,2,-0.000+0.001j
797,1,3,1,2,2,4,1,5,3,-0.628-3.342j
798,1,3,1,2,2,4,1,5,4,-0.002-0.002j


In [7]:
k, p, r = (4, 1, 5)

In [8]:
res_filtered = multipyles.filter_results(results, {'atom': 0, 'k': k, 'p': p, 'r': r, 'nu': (k+p)%2})
res_filtered

,species,atom,nu,l1,l2,k,p,r,t,value
756,1,0,1,2,2,4,1,5,-5,0.001+0.001j
757,1,0,1,2,2,4,1,5,-4,-0.000-0.003j
758,1,0,1,2,2,4,1,5,-3,-0.628+3.342j
759,1,0,1,2,2,4,1,5,-2,-0.001+0.001j
760,1,0,1,2,2,4,1,5,-1,0.001+0.001j
761,1,0,1,2,2,4,1,5,0,-3.408+0.000j
762,1,0,1,2,2,4,1,5,1,-0.001+0.001j
763,1,0,1,2,2,4,1,5,2,-0.001-0.001j
764,1,0,1,2,2,4,1,5,3,0.628+3.342j
765,1,0,1,2,2,4,1,5,4,0.000+0.003j


In [9]:
multipyles.transform_results_to_real(res_filtered)

,species,atom,nu,l1,l2,k,p,r,t,value
756,1,0,1,2,2,4,1,5,-5,-0.001
757,1,0,1,2,2,4,1,5,-4,0.004
758,1,0,1,2,2,4,1,5,-3,-4.727
759,1,0,1,2,2,4,1,5,-2,-0.001
760,1,0,1,2,2,4,1,5,-1,-0.001
761,1,0,1,2,2,4,1,5,0,-3.408
762,1,0,1,2,2,4,1,5,1,0.002
763,1,0,1,2,2,4,1,5,2,-0.001
764,1,0,1,2,2,4,1,5,3,-0.888
765,1,0,1,2,2,4,1,5,4,0.000


In [10]:
with open('tests/Cr2O3_benchmark_vasp/TENSMOM.R1.OUT', 'r') as file:
    res_vasp = multipyles.filter_results(read_from_dft.read_multipoles_from_vasp(file), {'atom': 1, 'l1': 2, 'l2': 2,
                                                                                         'k': k, 'p': p, 'r': r, 'nu': (k+p)%2})

In [11]:
res_vasp

,species,atom,nu,l1,l2,k,p,r,t,value
637,1,1,1,2,2,4,1,5,-5,0.000-0.000j
638,1,1,1,2,2,4,1,5,-4,-0.000-0.002j
639,1,1,1,2,2,4,1,5,-3,-0.627+3.343j
640,1,1,1,2,2,4,1,5,-2,-0.000+0.001j
641,1,1,1,2,2,4,1,5,-1,0.001-0.000j
642,1,1,1,2,2,4,1,5,0,-3.407+0.000j
643,1,1,1,2,2,4,1,5,1,-0.001-0.000j
644,1,1,1,2,2,4,1,5,2,-0.000-0.001j
645,1,1,1,2,2,4,1,5,3,0.627+3.343j
646,1,1,1,2,2,4,1,5,4,-0.000+0.002j


In [12]:
multipyles.transform_results_to_real(res_vasp)

,species,atom,nu,l1,l2,k,p,r,t,value
637,1,1,1,2,2,4,1,5,-5,0.000
638,1,1,1,2,2,4,1,5,-4,0.003
639,1,1,1,2,2,4,1,5,-3,-4.727
640,1,1,1,2,2,4,1,5,-2,-0.001
641,1,1,1,2,2,4,1,5,-1,0.000
642,1,1,1,2,2,4,1,5,0,-3.407
643,1,1,1,2,2,4,1,5,1,0.002
644,1,1,1,2,2,4,1,5,2,-0.000
645,1,1,1,2,2,4,1,5,3,-0.887
646,1,1,1,2,2,4,1,5,4,-0.000


Indeed, the multipoles from this code are the same as calculated in VASP directly.

# Calculates exchange and Hartree energy

In [13]:
xc_energies = multipyles.calculate_hartree_and_exchange_energies(l, results, uj=(2, 0))

Calculated Slater integrals = (2, 0.0, 0.0)


In [14]:
xc_energies

,species,atom,nu,l1,l2,k,p,r,w.w,exchange F0,exchange F2,exchange F4,hartree F0,hartree F2,hartree F4,exchange total,hartree total
0,1,0,0,2,2,0,0,0,12.298,-0.615,-0.176,-0.176,6.149,0.000,0.000,-1.230,12.298
1,1,0,0,2,2,0,1,1,0.000,-0.000,-0.000,-0.000,0.000,0.000,0.000,0.000,0.000
2,1,0,0,2,2,1,0,1,0.000,-0.000,-0.000,0.000,0.000,0.000,0.000,-0.000,0.000
3,1,0,0,2,2,1,1,0,0.008,-0.000,-0.000,0.000,0.000,0.000,0.000,-0.001,0.000
4,1,0,0,2,2,1,1,1,0.000,-0.000,-0.000,0.000,0.000,0.000,0.000,-0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,1,3,1,2,2,3,1,4,0.000,-0.000,0.000,0.000,0.000,0.000,0.000,-0.000,0.000
140,1,3,1,2,2,4,0,4,0.000,-0.000,-0.000,-0.000,0.000,0.000,0.000,-0.000,0.000
141,1,3,1,2,2,4,1,3,30.108,-0.048,-0.004,-0.000,0.000,0.000,0.000,-0.096,0.000
142,1,3,1,2,2,4,1,4,16.854,-0.061,-0.005,-0.000,0.000,0.000,0.000,-0.122,0.000
